4 December 2023
Doing a prototype with strictly defined data for both Ocado and Sainsburys.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import data_extraction
import re
db_extract = data_extraction.DataExtractor()
from fuzzywuzzy import fuzz, process

In [ ]:
o_df = pd.read_excel('Ocado and Sainsbury data.xlsx', sheet_name='Sheet2')
o_df.columns = ['label', 'generic_product', 'quantity']
o_df['generic_product'] = df['generic_product'].str.replace(' ','%20')
ocado_common_url = 'https://www.ocado.com/search?entry='
ocado_data = []
for value in o_df['generic_product']:
    url = ocado_common_url + value
    o_new_df = db_extract.extract_ocado_info(url)
    o_new_df['generic_product'] = value
    ocado_data.append(o_new_df)
    o_final_df = pd.concat(ocado_data, ignore_index=True)




In [ ]:
clean_df = o_final_df[~o_final_df['product_names'].str.contains('M&S')]
# clean_df['delivered_prices'] = clean_df['delivered_prices'].replace('[^0-9.]','',regex=True)
master_df = pd.merge(df, clean_df, on='generic_product')
master_df['generic_product'] = master_df['generic_product'].str.replace('%20',' ')
for var in ['product_names', 'weights']:
    master_df[var] = master_df[var].str.upper()
master_df['product_names'] = master_df['product_names'] + ' ' + master_df['weights']
master_df.drop('weights', axis=1, inplace=True)
def calculate_similarity(row):
    return fuzz.ratio(row['generic_product'], row['product_names'])
# Apply the function to each row in the DataFrame
master_df['similarity'] = master_df.apply(calculate_similarity, axis=1)

sorted_df = master_df.sort_values(by=['generic_product', 'similarity'], ascending = False)
sorted_df = sorted_df[~sorted_df['product_names'].str.contains('Longlife')]
# Get the index of the maximum similarity score within each group
max_similarity_indexes = sorted_df.groupby('generic_product')['similarity'].idxmax()
# Retain only the rows with the maximum similarity score for each group
ocado_df = sorted_df.loc[max_similarity_indexes]
ocado_df.head(30)



Now doing the same for Sainsburys data !!!

In [ ]:

df = pd.read_excel('Ocado and Sainsbury data.xlsx', sheet_name='Sheet2')
df.columns = ['label', 'generic_product', 'quantity']
df['generic_product'] = df['generic_product'].str.replace(' ','%20')
sainsbury_common_url = 'https://www.sainsburys.co.uk/gol-ui/SearchResults/'
sainsbury_data = []
for value in df['generic_product']:
    url = sainsbury_common_url + value
    new_df = db_extract.extract_sainsbury_info(url)
    new_df['generic_product'] = value
    sainsbury_data.append(new_df)
    final_df = pd.concat(sainsbury_data, ignore_index=True)


In [ ]:
final_df.info()

In [ ]:

clean_df = final_df[~final_df['product_names'].str.contains('Longlife')]
#clean_df['pre_nectar_price'] = clean_df['pre_nectar_price'].replace('[^0-9.]','',regex=True)
master_df = pd.merge(df, clean_df, on='generic_product')
master_df['generic_product'] = master_df['generic_product'].str.replace('%20',' ')
for var in ['product_names']:
    master_df[var] = master_df[var].str.upper()

master_df['product_names'] = master_df['product_names'].str.replace('TASTE THE DIFFERENCE','')

def calculate_similarity(row):
    return fuzz.ratio(row['generic_product'], row['product_names'])
# Apply the function to each row in the DataFrame
master_df['similarity'] = master_df.apply(calculate_similarity, axis=1)

sorted_df = master_df.sort_values(by=['generic_product', 'similarity'], ascending = False)
# Get the index of the maximum similarity score within each group
max_similarity_indexes = sorted_df.groupby('generic_product')['similarity'].idxmax()
# Retain only the rows with the maximum similarity score for each group
sainsbury_df = sorted_df.loc[max_similarity_indexes]
sainsbury_df.rename(columns={'product_names': 'sains_product_names'}, inplace=True)
# sainsbury_df.head(30)


In [ ]:
merged_df = pd.merge(ocado_df, sainsbury_df, on='generic_product')
merged_df.drop(['label_y', 'quantity_y','similarity_x', 'similarity_y'], axis=1, inplace=True)

In [ ]:
def add_hash(value):
    if value.startswith('£'):
        return value
    else: 
        return '0.' + value

# Apply the function to 'Column1'
for var in ['delivered_prices', 'pre_nectar_price']:
    merged_df[var] = merged_df[var].apply(add_hash)
    merged_df[var] = merged_df[var].replace('[^0-9.]','',regex=True)
    merged_df[var] = merged_df[var].astype(float)

merged_df.head(50)

In [ ]:
merged_df['ocado_value'] = merged_df['quantity_x'] * merged_df['delivered_prices']
merged_df['sains_value'] = merged_df['quantity_x'] * merged_df['pre_nectar_price']
ocado_order_value = merged_df['ocado_value'].sum()
sains_order_value = merged_df['sains_value'].sum()
date = '4 December 2023'

print(f'Order at Ocado costs {ocado_order_value}')
print(f'Order at Sainsbury costs {sains_order_value}')

In [95]:
date = '4 December 2023'
analysis_df = pd.DataFrame({
            'ocado cost': ocado_order_value,
            'sainsbury_pre_nectar': sains_order_value,
            'date_recorded':  date
            }, index=[0])

analysis_df.to_csv('Master_Analysis.csv') # Place all these into an excel folder separately

In [97]:
merged_df.head(50)
print(f'Order at Ocado costs {ocado_order_value}')
print(f'Order at Sainsbury costs {sains_order_value}')

Order at Ocado costs 57.85
Order at Sainsbury costs 58.66


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
trolley_url = 'https://www.sainsburys.co.uk/gol-ui/SearchResults/fairy%20dishwasher'
driver = webdriver.Chrome()
driver.get(trolley_url)



In [12]:
main_grid = driver.find_element(By.XPATH, value = '//ul[@class="ln-o-grid ln-o-grid--matrix ln-o-grid--equal-height"]')
main_price = main_grid.find_elements(By.XPATH, value = '//div[@class="pt__cost"]')
prices = []
for x in range(len(main_price)):
    price = main_price[x].text
    prices.append(price)

print(prices)

['£9.00with Nectar\n£12.00£15.96 / kg', '£5.00with Nectar\n£6.50£17.91 / kg', '£7.00with Nectar\n£9.00£17.54 / kg', '£12.00£11.76 / kg', '£10.00£8.83 / kg', '£8.00£9.52 / kg', '£13.50£12.77 / kg', '£12.00£16.13 / kg', '£4.00£13.03 / kg', '£15.00£12.56 / kg', '£9.00£16.27 / kg', '£9.00£17.34 / kg', '£13.00£15.13 / kg', '£5.60£6.67 / kg', '£10.00with Nectar\n£13.80£18.65 / kg', '£10.00£7.75 / kg', '£10.00with Nectar\n£12.90£15.00 / kg', '£6.00£12.00 / kg', '£4.00£8.33 / kg', '£3.25£12.90 / kg', '£13.00£7.22 / kg', '£13.80£15.68 / kg', '£3.30£3.30 / kg', '£6.00£9.15 / kg', '£5.00£11.11 / kg', '£8.00with Nectar\n£12.60£22.11 / kg', '£8.00£14.21 / kg', '£7.80£15.60 / kg', '£1.49£4.97 / kg', '£8.00£8.89 / kg', '£5.00with Nectar\n£7.60£27.14 / kg', '£12.65£26.35 / kg', '£17.00£12.50 / kg', '£5.00with Nectar\n£7.60£24.52 / kg', '£7.00£137.25 / kg', '£3.10£7.38 / kg', '£5.00with Nectar\n£7.60£28.57 / kg', '£14.00£21.94 / kg', '£8.80£17.60 / kg', '£1.50£250.00 / ltr', '£5.00£5.75 / ltr', '£3.00w